Read FiftyDatasets/AllDatasetsInfo/AllDataSourcesInfo/Kaggle/Sato/ Viznet datasets

In [1]:
import pandas as pd
from datetime import datetime
import re  

DB_or_dataset = input("Insert if using Datasets (d), Database.tables (dt), the two Data Sources (ds), Kaggle Datasets (k), Kaggle Datasets to correct (kc), Sato Datasets (s), Sato Filtered (sf), Filtered Viznet Datasets (v), or Filtered Kaggle Viznet Datasets (kv): ")

print(DB_or_dataset)

if DB_or_dataset.lower() == 'd':
    print("Using Datasets")
    datasets_xlsx = pd.read_excel("FiftyDatasets.xlsx") 
    columns_xlsx = pd.read_excel("AllColumnsFromFiftyDatasets.xlsx")
    analysed_columns_file_path = 'AnalysedColumns.xlsx'     
elif DB_or_dataset.lower() == 'dt':
    print("Using Database tables")
    datasets_xlsx = pd.read_excel("AllDatasetsInfo.xlsx")
    columns_xlsx = pd.read_excel("AllColumnsInfo.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsDB.xlsx'
elif DB_or_dataset.lower() == 'ds':
    print("Using Datasets and Database tables")
    datasets_xlsx = pd.read_excel("AllDataSourcesInfo.xlsx")
    columns_xlsx = pd.read_excel("AllAttributes_andColumnsInfo.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsDS.xlsx'
elif DB_or_dataset.lower() == 'k':
    print("Using Kaggle Datasets")
    datasets_xlsx = pd.read_excel("kaggle_datasets_with_domain_and_match.xlsx")
    columns_xlsx = pd.read_excel("kaggle_headers.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsK.xlsx'
elif DB_or_dataset.lower() == 'kc':
    print("Using Kaggle Datasets Corrections")
    datasets_xlsx = pd.read_excel("kaggle_datasets_with_domain_and_match.xlsx")
    columns_xlsx = pd.read_excel("kaggle_headers_10k.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsKc.xlsx'
elif DB_or_dataset.lower() == 's':
    print("Using Sato Datasets")
    datasets_xlsx = pd.read_excel("datasets_viznet.xlsx")
    columns_xlsx = pd.read_excel("columns_sato_only.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsSato.xlsx'
elif DB_or_dataset.lower() == 'sf':
    print("Using Sato Filtered Datasets")
    datasets_xlsx = pd.read_excel("datasets_viznet.xlsx")
    columns_xlsx = pd.read_excel("filtered_columns_sato_only.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsSatoFiltered.xlsx'
elif DB_or_dataset.lower() == 'v':
    print("Using Viznet Filtered Datasets")
    datasets_xlsx = pd.read_excel("datasets_viznet.xlsx")
    columns_xlsx = pd.read_excel("filtered_columns_viznet_all.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsViznetFiltered.xlsx'
elif DB_or_dataset.lower() == 'kv':
    print("Using Kaggle Viznet Filtered Datasets")
    datasets_xlsx = pd.read_excel("kaggle_viznet_datasets.xlsx")
    columns_xlsx = pd.read_excel("filtered_kaggle_viznet_headers.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsKaggleViznetFiltered.xlsx'
elif DB_or_dataset.lower() == 'sem':
    print("Using SemTabTest 2024 terms")
    datasets_xlsx = pd.read_excel("SemTabTest2024Dataset.xlsx")
    columns_xlsx = pd.read_excel("SemTabTest2024.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsSemTabTest2024.xlsx'
elif DB_or_dataset.lower() == 'dbp':
    print("Using DbPedia metadata")
    datasets_xlsx = pd.read_excel("DbpediaDataset.xlsx")
    columns_xlsx = pd.read_excel("DbpediaMetadataColumns.xlsx")
    analysed_columns_file_path = 'AnalysedColumnsDbpediaMetadata.xlsx'
else:
    raise ValueError("Invalid input. Please enter 'd', 'dt', 'ds', 'k', 'kc', 's', 'sf', 'v' or 'kv'.")



sem
Using SemTabTest 2024 terms


In [2]:
columns_xlsx = columns_xlsx.rename(columns={'index': 'dataset_index'})

columns_xlsx.head()

,dataset_index,name,area,Original Column,ID,Column,Path_Column,URL
0,r1_test_metadata.jsonl,SemTabTest,SemTab2024,1. ALT(m),1,ALT(m),1146722_1_7558140036342906956_ALT(m),http://dbpedia.org/ontology/elevation
1,r1_test_metadata.jsonl,SemTabTest,SemTab2024,2. Endroit,2,Endroit,11599512_1_280388135214354946_Endroit,http://dbpedia.org/ontology/region
2,r1_test_metadata.jsonl,SemTabTest,SemTab2024,3. AUReleaseDate,3,AUReleaseDate,11833461_1_3811022039809817402_AUReleaseDate,http://dbpedia.org/ontology/releaseDate
3,r1_test_metadata.jsonl,SemTabTest,SemTab2024,4. EUReleaseDate,4,EUReleaseDate,11833461_1_3811022039809817402_EUReleaseDate,http://dbpedia.org/ontology/releaseDate
4,r1_test_metadata.jsonl,SemTabTest,SemTab2024,5. Publisher,5,Publisher,11833461_1_3811022039809817402_Publisher,http://dbpedia.org/ontology/publisher


Clean Columns

In [3]:
import pandas as pd
import re
from datetime import datetime

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    # 1. Extract the ID from the column name 
    df['ID'] = df['Original Column'].str.extract(r'(\d+)\.')

    # 2. Extract the Column name up to the first colon, slash, or parenthesis
    df['Column'] = df['Original Column'].str.extract(r'\d+\.\s*([^/(:]+)')

    # 3. Extract everything after the first colon, slash, or parenthesis as the full description
    df['Description'] = df['Original Column'].str.extract(r'\d+\.\s*[^/(:]+\s*[:(/](.*)')[0]

    # 4. For cases where Description is None,
    #    extract the content within parentheses
    mask = df['Description'].isnull()
    df.loc[mask, 'Description'] = df.loc[mask, 'Original Column'].str.extract(r'\(([^)]+)\)')

    # 5. Trim spaces from all string columns
    for col in df.columns:
        if df[col].dtype == "object" and col != 'dataset_index':
            df[col] = df[col].str.strip()

    # 6. Replace empty strings with pd.NA  ← changed from None to pd.NA to avoid downcasting warning
    df = df.replace(r'^\s*$', pd.NA, regex=True)
    df = df.infer_objects(copy=False)
    
    # 7. Ensure Description is object dtype before applying .str.replace  ← added to avoid incompatible dtype warning
    df['Description'] = df['Description'].astype(object)

    # 8. Only apply .str.replace on non-null Description rows,
    #    converting to string first to avoid errors
    mask_desc = df['Description'].notnull()
    df.loc[mask_desc, 'Description'] = (
        df.loc[mask_desc, 'Description']
          .astype(str)
          .str.replace(r'\(', ' ', regex=True)
          .str.replace(r'\)', '', regex=True)
    )

    # 9. Reorder columns to ensure ID comes before Column
    columns_order = ['dataset_index', 'name', 'area', 'Original Column', 'ID', 'Column', 'Description']
    columns_order = [col for col in columns_order if col in df.columns]
    df = df[columns_order]

    return df

# Test the function with a sample DataFrame
if __name__ == "__main__":
    test_df = pd.DataFrame({
        'Original Column': [
            '14. num (the predicted attribute)',
            '1. lettr: capital letter (26 values from A to Z)',
            '2. x-box: horizontal position of the box (integer)',
            '1. sepal length in cm',
            '20. Foreign worker (qualitative)',
            '12. A12: Categorical with values: t, f',
            '55. capital_run_length_average (1 continuous real attribute): Average length of uninterrupted sequences of capital letters',
            '1. Class Name (party affiliation): democrat, republican',
            '1.Sex / nominal / -- / M, F, and I (infant)',
            '3.Cell Nucleus 1 - a) radius (mean of distances from center to points on the perimeter)',
            '14. win_loc (varchar(255))',
            '2.   Country'
        ],
        'dataset_index': ['SATO_000001'] * 12,
        'name': [None] * 12,
        'area': ['Sato-Viznet'] * 12
    })

print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
cleaned_df = clean_columns(test_df)
cleaned_df

Last run on: 2025-11-07 21:44:15


,dataset_index,name,area,Original Column,ID,Column,Description
0,SATO_000001,None,Sato-Viznet,14. num (the predicted attribute),14,num,the predicted attribute
1,SATO_000001,None,Sato-Viznet,1. lettr: capital letter (26 values from A to Z),1,lettr,capital letter 26 values from A to Z
2,SATO_000001,None,Sato-Viznet,2. x-box: horizontal position of the box (inte...,2,x-box,horizontal position of the box integer
3,SATO_000001,None,Sato-Viznet,1. sepal length in cm,1,sepal length in cm,NaN
4,SATO_000001,None,Sato-Viznet,20. Foreign worker (qualitative),20,Foreign worker,qualitative
5,SATO_000001,None,Sato-Viznet,"12. A12: Categorical with values: t, f",12,A12,"Categorical with values: t, f"
6,SATO_000001,None,Sato-Viznet,55. capital_run_length_average (1 continuous r...,55,capital_run_length_average,1 continuous real attribute: Average length of...
7,SATO_000001,None,Sato-Viznet,"1. Class Name (party affiliation): democrat, r...",1,Class Name,"party affiliation: democrat, republican"
8,SATO_000001,None,Sato-Viznet,"1.Sex / nominal / -- / M, F, and I (infant)",1,Sex,"nominal / -- / M, F, and I infant"
9,SATO_000001,None,Sato-Viznet,3.Cell Nucleus 1 - a) radius (mean of distance...,3,Cell Nucleus 1 - a) radius,mean of distances from center to points on the...


Open formats and abreviations dictionaries

In [4]:
dictionary = {}

# Open the formats dictionary file and read line by line 
with open("formats_dictionary.txt", "r") as file:
    for line in file:
        # Remove the trailing newline and comma, then split the line into key and value at the colon
        key, value = line.rstrip(",\n").split(":")
    
        # Remove the quotes around the key and value
        key = key.strip("'").lower()
        value = value.strip("'")

        # Add the key-value pair to the dictionary
        dictionary[key] = value

# Load the abbreviations dictionary
abbreviations_dict = {}
with open("abbreviations_dictionary.txt", "r") as file:
    for line in file:
        abbr, full_form = line.strip().split(":")
        abbreviations_dict[abbr.strip().lower()] = full_form.strip()
        
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Last run on: 2025-11-07 21:44:15


Replace abbreviations

In [5]:
def replace_abbreviations(text, abbreviations_dict):
    tokens = re.findall(
        r'\b\w+(?:[-=]\w+)*\b'          # Words, with hyphens/equals (like 0-4, 1=NE)
        r'|[=]'                         # Standalone equal signs
        r'|[°º][CFK]'                   # °C, ºF, etc.
        r'|[°º\-]+'                     # Standalone degree/hyphens
        r'|[a-zA-Z]+/[a-zA-Z]+(?:[²³μµ]?)'   # mg/dL, kg/m², µg/m³, cm², etc.
        r'|\d+[²³]?'                    # 10², 5³ (numbers with superscripts)
        r'|%'                           # <- add this line to capture % as a token
        , text
    )
    replaced_tokens = [abbreviations_dict.get(token.lower(), token) for token in tokens]
    return " ".join(replaced_tokens)

target_words_dict = dictionary
description_words_dict = dictionary

test_df = pd.DataFrame({
    "Original Column": [
        "1. chlorides (mg/dL)",                # plural with unit
        "2. chloride (mg/dL)",                 # singular with unit
        "3. Fuel_Price (USD/L)",               # phrase with underscore, unit
        "4. Tumor-size: 0-4, 5-9, 10-14",      # category ranges with hyphens
        "5. zone: 1=NE, 2=SE, 3=SW, 4=NW",     # code marker with equals
        "6. temperature (°C)",                 # temperature symbol
        "7. temperature (ºF)",                 # alternate degree symbol
        "8. Tdewpoint (from Chievres weather station), °C", # degree symbol at end
        "9. cm² measurement",                  # superscript unit
        "10. workclass",                       # compound word
        "11. landmass",                        # compound word
        "12. thickness (mm³)",                 # unit with superscript ³
        "13. concentration: 5mg/dL",           # value with unit
        "14. value: 10cm³",                    # value with unit
        "15. intensity: 5³, 10²",              # numeric with superscripts
        "16. 0-1 category",                    # hyphen, short
        "17. 1=Red, 2=Blue, 3=Green",          # equals for codes
        "18. percent (%)",                     # symbol in column
        "19. velocity (km/h)",                 # unit with slash
        "20. area: 10cm2, 10cm³",              # unit, no symbol and symbol
        "21. Alcohol (%)",                     # percent
        "22. BMI (kg/m²)",                     # slash, superscript
        "23. air_quality_index (AQI)",         # abbreviation as token
        "24. mean-pH",                         # hyphen in column name
    ]
})

# Apply the abbreviation/tokenizer function to every row
test_df['AbbrExpanded'] = test_df['Original Column'].apply(lambda x: replace_abbreviations(str(x), abbreviations_dict))

# Display original and processed columns side by side for easy comparison
pd.set_option('display.max_colwidth', 200)
test_df[['Original Column', 'AbbrExpanded']]

#print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


,Original Column,AbbrExpanded
0,1. chlorides (mg/dL),1 chlorides mg dL
1,2. chloride (mg/dL),2 chloride mg dL
2,3. Fuel_Price (USD/L),3 Fuel_Price USD losses
3,"4. Tumor-size: 0-4, 5-9, 10-14",4 Tumor-size 0-4 5-9 10-14
4,"5. zone: 1=NE, 2=SE, 3=SW, 4=NW",5 zone 1=NE 2=SE 3=SW 4=NW
5,6. temperature (°C),6 temperature °C
6,7. temperature (ºF),7 temperature ºF
7,"8. Tdewpoint (from Chievres weather station), °C",8 Tdewpoint from Chievres weather station °C
8,9. cm² measurement,9 cm² measurement
9,10. workclass,10 workclass


Split Camel Case

In [6]:
import re

def split_camel_case(s):
    if s.lower() == 'ph':
        return 'ph'
    # Split on separators
    tokens = re.split(r'[\s._\-]+', s)
    processed_tokens = []

    for token in tokens:
        if not token:
            continue
        # If all uppercase or digits, keep as is
        if token.isupper() or token.isdigit():
            processed_tokens.append(token.lower())
            continue
        # If all-uppercase+digits, keep as is
        if re.match(r'^[A-Z]{2,}\d+$', token):
            processed_tokens.append(token.lower())
            continue

        # Character-by-character scan
        words = []
        current = ''
        i = 0
        while i < len(token):
            c = token[i]
            if current == '':
                current = c
            elif (
                # lower->upper transition
                (current[-1].islower() and c.isupper()) or
                # digit->letter or letter->digit
                (current[-1].isdigit() and c.isalpha()) or
                (current[-1].isalpha() and c.isdigit())
            ):
                words.append(current)
                current = c
            else:
                current += c
            i += 1
        if current:
            words.append(current)

        # Now check if the last two or more are all caps: join them!
        if len(words) >= 2 and all(w.isupper() for w in words[-2:]) and len(''.join(words[-2:])) >= 2:
            # Join last all-cap runs
            n = len(words) - 1
            while n > 0 and words[n].isupper():
                n -= 1
            # All uppercase words from n+1 to end are the tail
            head = words[:n+1]
            tail = ''.join(words[n+1:])
            processed_tokens.extend([w.lower() for w in head if w])
            if tail:
                processed_tokens.append(tail.lower())
        else:
            processed_tokens.extend([w.lower() for w in words if w])

    return ' '.join(processed_tokens)

print(split_camel_case("reviews.sourceURLs"))   # reviews source urls
print(split_camel_case("prices.sourceURLs"))    # prices source urls
print(split_camel_case("sourceURLs"))           # source urls
print(split_camel_case("UrlLength"))            # url length
print(split_camel_case("nation_flag_url"))      # nation flag url
print(split_camel_case("URLs"))              # mean ph
print(split_camel_case("JSONFileURL"))          # json file url
print(split_camel_case("ODIDs"))                # odids
print(split_camel_case("ODID"))                # odid


reviews source urls
prices source urls
source urls
url length
nation flag url
urls
jsonfile url
odids
odid


In [7]:
columns_xlsx

,dataset_index,name,area,Original Column,ID,Column,Path_Column,URL
0,r1_test_metadata.jsonl,SemTabTest,SemTab2024,1. ALT(m),1,ALT(m),1146722_1_7558140036342906956_ALT(m),http://dbpedia.org/ontology/elevation
1,r1_test_metadata.jsonl,SemTabTest,SemTab2024,2. Endroit,2,Endroit,11599512_1_280388135214354946_Endroit,http://dbpedia.org/ontology/region
2,r1_test_metadata.jsonl,SemTabTest,SemTab2024,3. AUReleaseDate,3,AUReleaseDate,11833461_1_3811022039809817402_AUReleaseDate,http://dbpedia.org/ontology/releaseDate
3,r1_test_metadata.jsonl,SemTabTest,SemTab2024,4. EUReleaseDate,4,EUReleaseDate,11833461_1_3811022039809817402_EUReleaseDate,http://dbpedia.org/ontology/releaseDate
4,r1_test_metadata.jsonl,SemTabTest,SemTab2024,5. Publisher,5,Publisher,11833461_1_3811022039809817402_Publisher,http://dbpedia.org/ontology/publisher
...,...,...,...,...,...,...,...,...
136,r1_test_metadata.jsonl,SemTabTest,SemTab2024,137. Alphabeticcode,137,Alphabeticcode,97941125_0_8220652154649529701_Alphabeticcode,http://dbpedia.org/ontology/currencyCode
137,r1_test_metadata.jsonl,SemTabTest,SemTab2024,138. Numericcode,138,Numericcode,97941125_0_8220652154649529701_Numericcode,http://dbpedia.org/ontology/code
138,r1_test_metadata.jsonl,SemTabTest,SemTab2024,139. Symbol,139,Symbol,97941125_0_8220652154649529701_Symbol,http://dbpedia.org/ontology/symbol
139,r1_test_metadata.jsonl,SemTabTest,SemTab2024,140. Spouse,140,Spouse,98312357_0_6395417306876145729_Spouse,http://dbpedia.org/ontology/spouse


In [8]:
cleaned_columns_df = clean_columns(columns_xlsx.copy())
cleaned_columns_df

,dataset_index,name,area,Original Column,ID,Column,Description
0,r1_test_metadata.jsonl,SemTabTest,SemTab2024,1. ALT(m),1,ALT,m
1,r1_test_metadata.jsonl,SemTabTest,SemTab2024,2. Endroit,2,Endroit,NaN
2,r1_test_metadata.jsonl,SemTabTest,SemTab2024,3. AUReleaseDate,3,AUReleaseDate,NaN
3,r1_test_metadata.jsonl,SemTabTest,SemTab2024,4. EUReleaseDate,4,EUReleaseDate,NaN
4,r1_test_metadata.jsonl,SemTabTest,SemTab2024,5. Publisher,5,Publisher,NaN
...,...,...,...,...,...,...,...
136,r1_test_metadata.jsonl,SemTabTest,SemTab2024,137. Alphabeticcode,137,Alphabeticcode,NaN
137,r1_test_metadata.jsonl,SemTabTest,SemTab2024,138. Numericcode,138,Numericcode,NaN
138,r1_test_metadata.jsonl,SemTabTest,SemTab2024,139. Symbol,139,Symbol,NaN
139,r1_test_metadata.jsonl,SemTabTest,SemTab2024,140. Spouse,140,Spouse,NaN


Preprocess Columns

In [9]:
def preprocess_columns(df):
        
    # Only convert "Column" to string type for non-null values and then to lowercase to create "CleanedColumn"
    mask = df['Column'].notna()

    df.loc[mask, 'Column'] = df.loc[mask, 'Column'].str.replace('#', 'number', regex=False)
    
    df.loc[mask, 'CleanedColumn'] = df.loc[mask, 'Column'].astype(str).apply(split_camel_case)
    
    # Remove '-' and '_' characters only for non-null values
    mask = df['CleanedColumn'].notna()
    df.loc[mask, 'CleanedColumn'] = df.loc[mask, 'CleanedColumn'].str.replace('[-_]', ' ', regex=True)

    # Apply abbreviations dictionary to Description
    mask = df['Description'].notna()
    df.loc[mask, 'Description'] = df.loc[mask, 'Description'].apply(lambda x: replace_abbreviations(x, abbreviations_dict))
 
    return df

# Clean the columns

cleaned_columns_df = clean_columns(columns_xlsx.copy())

# Preprocess the column names
preprocessed_columns_df = preprocess_columns(cleaned_columns_df)
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

preprocessed_columns_df



Last run on: 2025-11-07 21:44:15


,dataset_index,name,area,Original Column,ID,Column,Description,CleanedColumn
0,r1_test_metadata.jsonl,SemTabTest,SemTab2024,1. ALT(m),1,ALT,m,alt
1,r1_test_metadata.jsonl,SemTabTest,SemTab2024,2. Endroit,2,Endroit,NaN,endroit
2,r1_test_metadata.jsonl,SemTabTest,SemTab2024,3. AUReleaseDate,3,AUReleaseDate,NaN,aurelease date
3,r1_test_metadata.jsonl,SemTabTest,SemTab2024,4. EUReleaseDate,4,EUReleaseDate,NaN,eurelease date
4,r1_test_metadata.jsonl,SemTabTest,SemTab2024,5. Publisher,5,Publisher,NaN,publisher
...,...,...,...,...,...,...,...,...
136,r1_test_metadata.jsonl,SemTabTest,SemTab2024,137. Alphabeticcode,137,Alphabeticcode,NaN,alphabeticcode
137,r1_test_metadata.jsonl,SemTabTest,SemTab2024,138. Numericcode,138,Numericcode,NaN,numericcode
138,r1_test_metadata.jsonl,SemTabTest,SemTab2024,139. Symbol,139,Symbol,NaN,symbol
139,r1_test_metadata.jsonl,SemTabTest,SemTab2024,140. Spouse,140,Spouse,NaN,spouse


KG Matching

In [ ]:
import pandas as pd
import difflib
import numpy as np
from gensim.models.fasttext import load_facebook_vectors
from datetime import datetime  # you referenced datetime at the end

# =========================================================
# 1. Load DBpedia and Schema.org terms
# =========================================================
dbpedia_terms_df = pd.read_excel('dbpedia.xlsx')
# labels are kept lowercase for matching
dbpedia_labels = dbpedia_terms_df['cleaned_label'].astype(str).str.lower().tolist()
dbpedia_ids = dbpedia_terms_df['id'].astype(str).tolist()
dbpedia_label_to_id = dict(zip(dbpedia_labels, dbpedia_ids))

schema_terms_df = pd.read_excel('schema.xlsx')
schema_labels = schema_terms_df['cleaned_label'].astype(str).str.lower().tolist()
schema_ids = schema_terms_df['id'].astype(str).tolist()
schema_label_to_id = dict(zip(schema_labels, schema_ids))

print("Loaded DBpedia labels:", dbpedia_labels[:5], "...")
print("DBpedia label to id (sample):", {k: dbpedia_label_to_id[k] for k in dbpedia_labels[:3]})
print("Loaded Schema.org labels:", schema_labels[:5], "...")
print("Schema.org label to id (sample):", {k: schema_label_to_id[k] for k in schema_labels[:3]})

# =========================================================
# 1.1 map “tail in lowercase → real URI” for DBpedia
# =========================================================
dbpedia_lower_tail_to_uri = {}
for uri in dbpedia_ids:
    tail = uri.rsplit('/', 1)[-1]
    lower_tail = tail.lower()
    dbpedia_lower_tail_to_uri.setdefault(lower_tail, uri)


def normalise_dbpedia_uri_safely(uri: str) -> str:
    """
    If DBpedia has a lowercase version of this URI tail, return it.
    Otherwise keep the original URI (for things like dbo:Airport).
    """
    if not uri:
        return uri
    base, tail = uri.rsplit('/', 1)
    lower_tail = tail.lower()
    if lower_tail in dbpedia_lower_tail_to_uri:
        return dbpedia_lower_tail_to_uri[lower_tail]
    return uri


# =========================================================
# 2. Syntactic match
# =========================================================
def syntactic_match(header, labels_list, label_to_id, threshold=0.9, kg="dbpedia"):
    header_l = header.strip().lower()
    # exact
    if header_l in label_to_id:
        uri = label_to_id[header_l]
        if kg == "dbpedia":
            uri = normalise_dbpedia_uri_safely(uri)
        return uri

    # close
    matches = difflib.get_close_matches(header_l, labels_list, n=1, cutoff=threshold)
    if matches:
        uri = label_to_id[matches[0]]
        if kg == "dbpedia":
            uri = normalise_dbpedia_uri_safely(uri)
        return uri

    return None


# =========================================================
# 3. Load FastText
# =========================================================
fasttext_model = load_facebook_vectors('cc.en.300.bin')  # update path if needed
print("Gensim FastText model loaded. Dimension:", fasttext_model.vector_size)


def get_embedding(text):
    words = text.split()
    vectors = [fasttext_model[w] for w in words if w in fasttext_model]
    if not vectors:
        return np.zeros(fasttext_model.vector_size)
    return np.mean(vectors, axis=0)


print("Embedding for 'birth date':", get_embedding("birth date")[:5])


# =========================================================
# 4. Semantic match
#    IMPORTANT CHANGE: first check if the header is an exact label.
#    If yes, return that right away so we don’t overwrite it with a
#    capitalised near-duplicate.
# =========================================================
def semantic_match(header,
                   labels_list,
                   label_to_id,
                   embeddings_dict,
                   similarity_threshold=0.9,
                   kg="dbpedia"):
    header_l = header.strip().lower()

    # ---- NEW: respect exact ontology labels even in semantic mode ----
    if header_l in label_to_id:
        uri = label_to_id[header_l]
        if kg == "dbpedia":
            uri = normalise_dbpedia_uri_safely(uri)
        # we return 1.0 here just to signal “perfect”
        return uri, 1.0
    # ------------------------------------------------------------------

    header_emb = get_embedding(header_l)
    best_sim = -1.0
    best_label = None

    for label, emb in embeddings_dict.items():
        sim = np.dot(header_emb, emb) / (np.linalg.norm(header_emb) * np.linalg.norm(emb) + 1e-8)
        if sim > best_sim:
            best_sim = sim
            best_label = label

    if best_sim >= similarity_threshold and best_label is not None:
        uri = label_to_id[best_label]
        if kg == "dbpedia":
            uri = normalise_dbpedia_uri_safely(uri)
        return uri, best_sim

    return None, best_sim


# =========================================================
# 5. Precompute embeddings
# =========================================================
dbpedia_embeddings = {label: get_embedding(label) for label in dbpedia_labels}
schema_embeddings = {label: get_embedding(label) for label in schema_labels}

print("Sample DBpedia label embedding shape:", dbpedia_embeddings[dbpedia_labels[0]].shape)
print("First 5 dims of first embedding:", dbpedia_embeddings[dbpedia_labels[0]][:5])

# =========================================================
# 6. Quick tests
# =========================================================
test_headers = ['birth date', 'organization', 'assets', 'zip code', 'unknownfield']
for th in test_headers:
    dbp_syn = syntactic_match(th, dbpedia_labels, dbpedia_label_to_id, kg="dbpedia")
    print(f"Syntactic match (DBpedia) for '{th}':", dbp_syn)

    sch_syn = syntactic_match(th, schema_labels, schema_label_to_id, kg="schema")
    print(f"Syntactic match (Schema.org) for '{th}':", sch_syn)

# headers to test semantic matching
sem_headers = ['assets', 'amt', 'org unit', 'unknownfield', 'languages']
for sh in sem_headers:
    dbp_sem, dbp_sim = semantic_match(
        sh, dbpedia_labels, dbpedia_label_to_id, dbpedia_embeddings, kg="dbpedia"
    )
    print(f"Semantic match (DBpedia) for '{sh}':", dbp_sem, dbp_sim)

    sch_sem, sch_sim = semantic_match(
        sh, schema_labels, schema_label_to_id, schema_embeddings, kg="schema"
    )
    print(f"Semantic match (Schema.org) for '{sh}':", sch_sem, sch_sim)

print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Loaded DBpedia labels: ['a side', 'abbeychurch blessing', 'abbeychurch blessing charge', 'abbreviation', 'able to grind'] ...
DBpedia label to id (sample): {'a side': 'http://dbpedia.org/ontology/aSide', 'abbeychurch blessing': 'http://dbpedia.org/ontology/abbeychurchBlessing', 'abbeychurch blessing charge': 'http://dbpedia.org/ontology/abbeychurchBlessingCharge'}
Loaded Schema.org labels: ['3 d model', 'abdomen', 'about', 'about page', 'abridged'] ...
Schema.org label to id (sample): {'3 d model': 'https://schema.org/3DModel', 'abdomen': 'https://schema.org/Abdomen', 'about': 'https://schema.org/about'}
Gensim FastText model loaded. Dimension: 300
Embedding for 'birth date': [ 0.00283275  0.0443643  -0.06757186  0.06091432  0.00819952]
Sample DBpedia label embedding shape: (300,)
First 5 dims of first embedding: [ 0.01555624 -0.24466786  0.00117554 -0.002059   -0.03555111]
Syntactic match (DBpedia) for 'birth date': http://dbpedia.org/ontology/birthDate
Syntactic match (Schema.org) fo

Apply analysis

In [11]:
import pandas as pd
import re

def match_with_plural(token, dictionary):
    # Attempt direct match first
    if token in dictionary:
        return dictionary[token]
    # Check for 'ies' -> 'y' (families -> family)
    if token.endswith('ies') and len(token) > 3:
        singular = token[:-3] + 'y'
        if singular in dictionary:
            return dictionary[singular]
    # Check for 'es' -> '' (e.g., classes -> class)
    if token.endswith('es') and len(token) > 2:
        singular = token[:-2]
        if singular in dictionary:
            return dictionary[singular]
    # Check for 's' -> '' (e.g., chlorides -> chloride)
    if token.endswith('s') and len(token) > 2:
        singular = token[:-1]
        if singular in dictionary:
            return dictionary[singular]
    return None

def apply_analysis(df, target_words_dict, description_words_dict, abbreviations_dict, all_datasets_info):

    def get_special_name_format(table_name, column_name):
        special_cases = {
            'city': 'city',
            'country': 'country',
            'state': 'state',
            'province': 'state'  # province is treated as state
        }
        
        if DB_or_dataset.lower() == 'd':
            # For datasets, table_name is a number, so we don't check it
            if column_name.lower() == 'name':
                return 'name', target_words_dict['name']
        else:
            # For database tables, check if any special case is in the table name
            table_name_str = str(table_name).lower()
            for case, format_type in special_cases.items():
                if case in table_name_str and column_name.lower() == 'name':
                    return f"{format_type}_name", target_words_dict[format_type]
        
        return 'name', target_words_dict['name']
    
    # main code 
    # Check if PK/FK information is available
    has_pk_fk_info = 'primary_key' in all_datasets_info.columns and 'foreign_keys' in all_datasets_info.columns

    # Get primary and foreign keys for each table
    pk_fk_dict = {}
    if has_pk_fk_info:
        for _, row in all_datasets_info.iterrows():
            table_name = row['index']
            pk = row['primary_key'] if pd.notna(row['primary_key']) else None
            fks = row['foreign_keys'].split(', ') if pd.notna(row['foreign_keys']) else []
            fk_columns = [fk.split(' -> ')[0] for fk in fks]
            pk_fk_dict[table_name] = {'pk': pk, 'fks': fk_columns} 

    # 'formats_ordered_list' is a list of dictionary keys in the order they appear in 'formats_dictionary'
    formats_ordered_list = list(target_words_dict.keys())

    # Initialize new columns
    df['ColumnKeyword'] = None
    df['ColumnFormat'] = None
    df['DescriptionKeyword'] = None
    df['DescriptionFormat'] = None
    count = 0
   
    # Apply target words analysis
    for i, row in df.iterrows():
        count += 1
        if count % 1000 == 0:
            print(f"✅ Processed {count} rows...")

        std_col_name = row['CleanedColumn']
        col_name = row['Column']
        table_name = row['dataset_index']

        # Skip if 'CleanedColumn' is missing
        if pd.isnull(std_col_name):
            continue

        # Split camel case
        std_col_name = split_camel_case(std_col_name)

        # Check if the column is a primary key or foreign key
        if table_name in pk_fk_dict:
            if col_name == pk_fk_dict[table_name]['pk'] or col_name in pk_fk_dict[table_name]['fks']:
                df.at[i, 'ColumnKeyword'] = 'id'
                df.at[i, 'ColumnFormat'] = 'IDcolumn'
                continue

        found = False

        # Iterate through each word based on the order in 'formats_ordered_list'
        for word in formats_ordered_list:
            analysis = target_words_dict[word.lower()]
            # Special handling for 'name'
            if word == 'name':
                keyword, format_type = get_special_name_format(table_name, col_name)
                if keyword != 'name':
                    df.at[i, 'ColumnKeyword'] = keyword
                    df.at[i, 'ColumnFormat'] = format_type
                    found = True
                    break
                pattern = rf'({word})(?![\\w-])'
            # Special handling for uppercase 'ID' at the end of a column name
            elif word == 'id' and col_name.endswith('ID'):
                df.at[i, 'ColumnKeyword'] = 'id'
                df.at[i, 'ColumnFormat'] = target_words_dict.get('id', 'ID column')
                found = True
                break
            else:
                # General matching for other terms
                pattern = rf'\b{word}\b'

            # Search for the pattern in the CleanedColumn
            if re.search(pattern, std_col_name, re.IGNORECASE):
                df.at[i, 'ColumnKeyword'] = word
                df.at[i, 'ColumnFormat'] = analysis
                found = True
                break

        # If no match found, replace abbreviations and try again
        if not found:
            replaced_text = replace_abbreviations(std_col_name, abbreviations_dict)
            for word in formats_ordered_list:
                pattern = rf'\b{word}\b'
                if re.search(pattern, replaced_text, re.IGNORECASE):
                    df.at[i, 'ColumnKeyword'] = word
                    df.at[i, 'ColumnFormat'] = target_words_dict[word.lower()]
                    found = True
                    break


        # If still no match, check tokens only
        if not found:
            tokens = re.findall(r'\b\w+\b|%', row['CleanedColumn'].lower())
            for token in tokens:
                expanded_token = abbreviations_dict.get(token.lower(), None)
                # First check plural support for expanded_token
                if expanded_token:
                    match = match_with_plural(expanded_token, target_words_dict)
                    if match:
                        df.at[i, 'ColumnKeyword'] = expanded_token
                        df.at[i, 'ColumnFormat'] = match
                        found = True
                        break
                # Then check plural support for the token itself
                match = match_with_plural(token, target_words_dict)
                if match:
                    df.at[i, 'ColumnKeyword'] = token
                    df.at[i, 'ColumnFormat'] = match
                    found = True
                    break

        # -- PRIORITIZE percentage if % present in CleanedColumn tokens --
        tokens = re.findall(r'\b\w+\b|%', row['CleanedColumn'].lower())
        if '%' in tokens and '%' in target_words_dict:
            df.at[i, 'ColumnKeyword'] = '%'
            df.at[i, 'ColumnFormat'] = target_words_dict['%']

    count = 0
    # Apply description words analysis
    for i, row in df.iterrows():
        count += 1
        if count % 1000 == 0:
            print(f"✅ Processed {count} rows...")
        # Skip if 'Description' is missing
        if pd.isnull(row['Description']):
            continue
        for word, analysis in description_words_dict.items():
            if word in ['is', 'has']:
                pattern = r'^\s*' + re.escape(word) + r'\b'
                if re.search(pattern, row['Description'], re.IGNORECASE):
                    df.at[i, 'DescriptionKeyword'] = word
                    df.at[i, 'DescriptionFormat'] = analysis
                    break
            else:
                if re.search(rf'\b{re.escape(word)}\b', row['Description'], re.IGNORECASE):
                    df.at[i, 'DescriptionKeyword'] = word
                    df.at[i, 'DescriptionFormat'] = analysis
                    break
                elif not word.isalnum():
                    if word.lower() in row['Description'].lower():
                        df.at[i, 'DescriptionKeyword'] = word
                        df.at[i, 'DescriptionFormat'] = analysis
                        break

    return df

# Apply this function to the dataframe
analysed_columns_df = apply_analysis(preprocessed_columns_df.copy(), target_words_dict, description_words_dict, abbreviations_dict, datasets_xlsx)


def match_with_ontology(
    header, labels_list, label_to_id, embeddings_dict,
    syntactic_thresh=0.9, semantic_thresh=0.7, column_keyword=None
):
    header_lc = header.lower().strip()

    # 1. Syntactic (exact) match
    match_id = syntactic_match(header_lc, labels_list, label_to_id, threshold=syntactic_thresh)
    if match_id:
        return match_id, 1.0

    # 2. Substring preference (force this to happen BEFORE semantic!)
    substr_matches = [label for label in labels_list if label in header_lc and len(label) >= 4]
    if substr_matches:
        substr_matches = sorted(
            substr_matches,
            key=lambda x: (len(x.split()), len(x)),  # more words, then longer
            reverse=True
        )
        best_label = substr_matches[0]
        print(f"[Substring wins] '{best_label}' in '{header}'")
        return label_to_id[best_label], 1.0

    # 3. Semantic match
    match_id, sim_score = semantic_match(header_lc, labels_list, label_to_id, embeddings_dict, similarity_threshold=semantic_thresh)
    if match_id:
        return match_id, sim_score

    # 4. ColumnKeyword
    if column_keyword:
        ck = column_keyword.lower().strip()
        match_id = syntactic_match(ck, labels_list, label_to_id, threshold=syntactic_thresh)
        if match_id:
            return match_id, 1.0
        match_id, sim_score_ck = semantic_match(ck, labels_list, label_to_id, embeddings_dict, similarity_threshold=semantic_thresh)
        if match_id:
            return match_id, sim_score_ck

    return "Nil", 0.0


def annotate_all_kgs(df, syntactic_thresh=0.9, semantic_thresh=0.75):
    def kg_annotate_row(row, labels_list, label_to_id, embeddings_dict):
        return match_with_ontology(
            str(row['CleanedColumn']),
            labels_list, label_to_id, embeddings_dict,
            syntactic_thresh, semantic_thresh,
            column_keyword=row.get('ColumnKeyword', None)
        )

    db_results = df.apply(
        lambda row: kg_annotate_row(row, dbpedia_labels, dbpedia_label_to_id, dbpedia_embeddings),
        axis=1
    )
    df['DBpediaType'] = [r[0] for r in db_results]
    df['DBpediaScore'] = [r[1] for r in db_results]

    schema_results = df.apply(
        lambda row: kg_annotate_row(row, schema_labels, schema_label_to_id, schema_embeddings),
        axis=1
    )
    df['SchemaOrgType'] = [r[0] for r in schema_results]
    df['SchemaScore'] = [r[1] for r in schema_results]

    return df

def get_top_k_ontology_matches(header, labels_list, label_to_id, embeddings_dict, k=5):
    header_lc = header.lower().strip()
    candidates = []

    # Syntactic (exact) match
    match_id = syntactic_match(header_lc, labels_list, label_to_id, threshold=0.9)
    if match_id:
        candidates.append((match_id, 1.0, 'syntactic'))

    # Semantic matches (top K by cosine)
    header_emb = get_embedding(header_lc)
    all_scores = []
    for label in labels_list:
        emb = embeddings_dict[label]
        score = np.dot(header_emb, emb) / (np.linalg.norm(header_emb) * np.linalg.norm(emb) + 1e-8)
        all_scores.append((label_to_id[label], score, 'semantic'))
    # Sort and take top k (not already present)
    top_sem = sorted(all_scores, key=lambda x: x[1], reverse=True)
    for uri, score, method in top_sem:
        if uri not in [c[0] for c in candidates]:
            candidates.append((uri, score, method))
        if len(candidates) >= k:
            break

    # Substring matches (if not already present)
    substr_matches = [label for label in labels_list if label in header_lc]
    for label in substr_matches:
        uri = label_to_id[label]
        if uri not in [c[0] for c in candidates]:
            candidates.append((uri, 0.98, 'substring'))
        if len(candidates) >= k:
            break

    # Only keep the first k, sorted by score (descending)
    candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]
    return candidates


analysed_columns_df['ColumnDescCombined'] = (
    analysed_columns_df['CleanedColumn'].fillna('').astype(str) + ' ' +
    analysed_columns_df['Description'].fillna('').astype(str)
)

analysed_columns_df = annotate_all_kgs(analysed_columns_df, syntactic_thresh=1.0)

analysed_columns_df['DBpediaTop5'] = analysed_columns_df['CleanedColumn'].apply(
    lambda x: get_top_k_ontology_matches(
        str(x), dbpedia_labels, dbpedia_label_to_id, dbpedia_embeddings, k=5
    )
)

# Preview results
ac = analysed_columns_df[['CleanedColumn', 'Description', 'DBpediaType', 'DBpediaScore', 'SchemaOrgType', 'SchemaScore']].head(60)

print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

ac

[Substring wins] 'release date' in 'aurelease date'
[Substring wins] 'release date' in 'eurelease date'
[Substring wins] 'height' in 'heightinmeters'
[Substring wins] 'code' in 'alphabeticcode'
[Substring wins] 'code' in 'numericcode'
[Substring wins] 'institution' in 'granting institution'
[Substring wins] 'ratio' in 'rationale'
[Substring wins] 'code' in 'alphabeticcode'
[Substring wins] 'code' in 'numericcode'
[Substring wins] 'diocese' in 'dioceses'
[Substring wins] 'height' in 'heightinmeters'
[Substring wins] 'code' in 'alphabeticcode'
[Substring wins] 'code' in 'numericcode'
[Substring wins] 'language' in 'languages'
[Substring wins] 'language' in 'languages'
[Substring wins] 'language' in 'languages'
[Substring wins] 'water' in 'watergauge'
[Substring wins] 'area' in 'landarea'
[Substring wins] 'founder' in 'founders'
[Substring wins] 'code' in 'alphabeticcode'
[Substring wins] 'code' in 'numericcode'
[Substring wins] 'release date' in 'aurelease date'
[Substring wins] 'release

,CleanedColumn,Description,DBpediaType,DBpediaScore,SchemaOrgType,SchemaScore
0,alt,m,http://dbpedia.org/ontology/altitude,1.000000,Nil,0.000000
1,endroit,NaN,Nil,0.000000,Nil,0.000000
2,aurelease date,NaN,http://dbpedia.org/ontology/releaseDate,1.000000,https://schema.org/releaseDate,1.000000
3,eurelease date,NaN,http://dbpedia.org/ontology/releaseDate,1.000000,https://schema.org/releaseDate,1.000000
4,publisher,NaN,http://dbpedia.org/ontology/publisher,1.000000,http://dbpedia.org/ontology/publisher,1.000000
5,address,NaN,http://dbpedia.org/ontology/address,1.000000,http://dbpedia.org/ontology/address,1.000000
6,format,NaN,http://dbpedia.org/ontology/format,1.000000,https://schema.org/DigitalFormat,0.863042
7,currency code,NaN,http://dbpedia.org/ontology/currencyCode,1.000000,https://schema.org/currency,1.000000
8,heightinmeters,NaN,http://dbpedia.org/ontology/height,1.000000,https://schema.org/height,1.000000
9,director,NaN,http://dbpedia.org/ontology/director,1.000000,http://dbpedia.org/ontology/director,1.000000


In [12]:
header = "au release date"
matches = [label for label in dbpedia_labels if (label in header or header in label) and len(label) >= 4]
matches = sorted(matches, key=lambda x: (len(x.split()), len(x)), reverse=True)
print(matches)

['release date', 'date']


FinalFormat

In [13]:
def analysis_of_column(ColumnFormat, DescriptionFormat, DescriptionKeyword, index_value):
    if ColumnFormat is None and DescriptionFormat is None:
        return 'NaN'

    # If either is 'None', choose the one that has a value
    if pd.isnull(ColumnFormat):
        return DescriptionFormat
    elif pd.isnull(DescriptionFormat):
        return ColumnFormat
  
    # Prioritize 'categorical' over 'numerical', but prefer specific numeric types in DescriptionKeyword
    if ColumnFormat == 'categorical' and DescriptionFormat == 'numerical':
        if DescriptionKeyword in ['float', 'double']:
            return DescriptionFormat
        return ColumnFormat
    
    # If one is 'string' and the other is not, choose the one that is not 'string'
    if ColumnFormat == 'string' and DescriptionFormat not in [None, 'string', 'NaN']:
        return DescriptionFormat
    elif DescriptionFormat == 'string' and ColumnFormat not in [None, 'string', 'NaN']:
        return ColumnFormat

    # If one is 'hour' and the other is numerical, choose numerical'
    if ColumnFormat == 'hour' and DescriptionFormat.startswith("numerical"):
        return DescriptionFormat
    elif DescriptionFormat == 'hour' and ColumnFormat.startswith("numerical"):
        return ColumnFormat
    
    # Datetime should always win
    if ColumnFormat == 'datetime' or DescriptionFormat == 'datetime':
        return 'datetime'

    # ID column should always win
    if ColumnFormat == "IDcolumn" or DescriptionFormat == "IDcolumn":
        return "IDcolumn"
    
    # percentage should always win
    if ColumnFormat == "percentage" or DescriptionFormat == "percentage":
        return "percentage"
    
    # binary should always win
    if ColumnFormat == 'binary' or DescriptionFormat == 'binary':
        return 'binary' 

    # If both are 'numerical' or if ColumnFormat is in the predefined list, return ColumnFormat
    if (ColumnFormat.startswith("numerical") and DescriptionFormat.startswith("numerical")) or \
       ColumnFormat in ["phone", "month", "date", "weekday", "week", "year",
                        "country", "state", "city", "street", "name",
                        "latitude", "longitude", "postalcode", "URLformat","IPformat", 
                        "E-mailformat", "binary"]:
        return ColumnFormat
    elif ColumnFormat in ["age"] and DescriptionFormat not in ['categorical']:
        return ColumnFormat
    # If no exceptions apply, return the value from "DescriptionFormat"
    else:
        return DescriptionFormat

# Apply the function to create the new column
analysed_columns_df['FinalFormat'] = analysed_columns_df.apply(
    lambda row: analysis_of_column(
        row['ColumnFormat'],
        row['DescriptionFormat'],
        row['DescriptionKeyword'],
        row['dataset_index']),  
    axis=1
)
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Last run on: 2025-11-07 21:49:54


Identify Origin

In [14]:
def identify_origin(row):
    analysis_col = row['FinalFormat']
    ColumnFormat = row['ColumnFormat']
    DescriptionFormat = row['DescriptionFormat']

    # Check if the value in 'FinalFormat' came from 'ColumnFormat'
    if pd.notnull(ColumnFormat) and analysis_col == ColumnFormat:
        return row['ColumnKeyword']

    # Check if the value in 'FinalFormat' came from 'DescriptionFormat'
    elif pd.notnull(DescriptionFormat) and analysis_col == DescriptionFormat:
        return row['DescriptionKeyword']

    # If none of the above conditions are met, return NaN or any default value you prefer
    else:
        return 'NaN'

# Create the new column 'SourceKeyword' in analysed_columns_df
analysed_columns_df['SourceKeyword'] = analysed_columns_df.apply(identify_origin, axis=1)
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Last run on: 2025-11-07 21:49:54


Save results

In [15]:
# Save the results to a new Excel file
analysed_columns_df.to_excel(analysed_columns_file_path, index=False)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Last run on: 2025-11-07 21:49:54


SemTab 2024 Metadata only track

In [16]:
import pandas as pd
import ast

# === 1. Read your result/output file and GT file ===
results_df = pd.read_excel('AnalysedColumnsSemTabTest2024.xlsx')
gt_df = pd.read_excel('r1_test_metadata_GT.xlsx', header=None)

# === 2. Helper to get column content after number+dot, preserving everything else ===
def extract_col_after_number(s):
    s = str(s)
    if '.' in s:
        return s.split('.', 1)[1].lstrip()  # Only removes leading space after the dot
    return s

# === 3. Build GT map: {column_name as in GT : dbpedia uri} ===
gt_map = {}
for i, row in gt_df.iterrows():
    gt_col_name = str(row[2]).strip('\n\r')  # Only strip newlines, not spaces or dots
    gt_dbpedia = str(row[1]).strip()
    gt_map[gt_col_name] = gt_dbpedia

# === 4. Main matching and output rows ===
excel_rows = []
top1_correct = 0
top5_correct = 0
total = 0

for i, row in results_df.iterrows():
    orig_col = str(row['Original Column'])
    col_key = extract_col_after_number(orig_col)

    gt_uri = gt_map.get(col_key, None)

    if gt_uri is None or gt_uri.lower() in ('', 'nan'):
        result = {
            "Original Column": orig_col,
            "Cleaned Column": col_key,
            "GT DBpediaType": None,
            "Model DBpediaType": row['DBpediaType'],
            "Model DBpediaTop5": None,
            "Top1?": False,
            "Top5?": False
        }
        excel_rows.append(result)
        continue

    total += 1
    # Top 1
    model_top1 = str(row['DBpediaType']).strip()
    top1 = model_top1 == gt_uri

    # Top 5
    found_top5 = False
    top5_uris = []
    top5_str = str(row['DBpediaTop5'])
    try:
        top5_list = ast.literal_eval(top5_str) if top5_str not in ('', 'nan', 'None') else []
        top5_uris = [entry[0] if isinstance(entry, (list, tuple)) else entry for entry in top5_list]
        if gt_uri in top5_uris:
            found_top5 = True
    except Exception as e:
        pass

    if top1: top1_correct += 1
    if found_top5: top5_correct += 1

    # Record row for Excel
    excel_rows.append({
        "Original Column": orig_col,
        "Cleaned Column": col_key,
        "GT DBpediaType": gt_uri,
        "Model DBpediaType": model_top1,
        "Model DBpediaTop5": top5_uris,
        "Top1?": top1,
        "Top5?": found_top5
    })

# === 5. Add summary row ===
output_df = pd.DataFrame(excel_rows)
summary_row = {
    "Original Column": "SUMMARY",
    "GT DBpediaType": f"Top1: {top1_correct}/{total} ({top1_correct/total:.2%})",
    "Model DBpediaType": f"Top5: {top5_correct}/{total} ({top5_correct/total:.2%})"
}
output_df = pd.concat([output_df, pd.DataFrame([summary_row])], ignore_index=True)

# === 6. Write to Excel, NO extra Excel libraries required ===
output_xlsx = 'dbpedia_match_results.xlsx'
output_df.to_excel(output_xlsx, index=False)

print(f"Results written to {output_xlsx}")

# ========== END OF SCRIPT ==========
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Results written to dbpedia_match_results.xlsx
Last run on: 2025-11-07 21:49:54
